# oceanarray demo - build climatology

This notebook walks through the method to build a climatology for the vertical interpolation


In [ ]:
from pathlib import Path
import numpy as np
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from oceanarray import readers, plotters, tools, convertOS, writers, mooring, rapid_interp


## Load hydrographic data

In [ ]:
data_dir = Path("..", "data")
hydro_data_path = data_dir / "training_data_WB2.nc"

ds_hydro = xr.open_dataset(hydro_data_path)
ds_hydro

### Subsample to 20dbar

In [ ]:

pmax = float(ds_hydro.PRES.max())
print("Maximum pressure in ds_interp:", pmax)

pmax = max(ds_hydro.PRES.max(), 3900)
print("Maximum pressure after adjustment:", pmax)

pgrid = rapid_interp.spacing(0, pmax, 20)
ds_standard = tools.process_dataset(ds_hydro, pgrid=pgrid)
ds_standard


### Calculate a climatology

In [ ]:
# Build the climatology
clim_ds = rapid_interp.build_climatology(ds_standard, pgrid)

# Smooth the climatology
clim_ds_smoothed = rapid_interp.smooth_climatology(clim_ds, window=3)

# Plot the climatology
fig, ax = plt.subplots(1, 2, figsize=(12, 3))
plotters.plot_climatology(clim_ds, clim_ds_smoothed=clim_ds_smoothed,var="dTdp", fig=fig, ax=ax[0])
plotters.plot_climatology(clim_ds, clim_ds_smoothed=clim_ds_smoothed, var="dSdp", fig=fig, ax=ax[1])

In [ ]:
## Fill the gaps in the climatology
clim_ds_smoothed

# Plot the seasonal cycle at each temperature bin for dTdp and dSdp
fig2, ax2 = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

for i, var in enumerate(['dTdp', 'dSdp']):
    # Each column is a temperature bin, each row is a month
    data = clim_ds_smoothed[var]
    for temp_idx, temp_val in enumerate(clim_ds_smoothed.TEMP.values):
        ax2[i].plot(clim_ds_smoothed['month'], data[:, temp_idx], alpha=0.3)
    ax2[i].set_title(f'Seasonal Cycle at Each Temperature Bin: {var}')
    ax2[i].set_xlabel('Month')
    ax2[i].set_ylabel(var)
    ax2[i].grid(True)

plt.tight_layout()
plt.show()

# At east temperature, linearly interpolate the climatology to fill gaps
for i, var in enumerate(['dTdp', 'dSdp']):
    clim_ds_smoothed[var] = clim_ds_smoothed[var].interpolate_na(dim='TEMP', method='linear')
# Plot the filled climatology
fig3, ax3 = plt.subplots(1, 2, figsize=(12, 3))
plotters.plot_climatology(clim_ds_smoothed, var="dTdp", fig=fig3, ax=ax3[0])
plotters.plot_climatology(clim_ds_smoothed, var="dSdp", fig=fig3, ax=ax3[1])
plt.tight_layout()
plt.show()

# For temperatures below 4 degrees, replace the seasonal cycle with the average climatology
for i, var in enumerate(['dTdp', 'dSdp']):
    mask = clim_ds_smoothed.TEMP < 4
    clim_ds_smoothed[var] = clim_ds_smoothed[var].where(~mask, clim_ds_smoothed[var].mean(dim='month'))
# Plot the final climatology
fig4, ax4 = plt.subplots(1, 2, figsize=(12, 3))
plotters.plot_climatology(clim_ds_smoothed, var="dTdp", fig=fig4, ax=ax4[0])
plotters.plot_climatology(clim_ds_smoothed, var="dSdp", fig=fig4, ax=ax4[1])
plt.tight_layout()
plt.show()

In [ ]:
### Save the climatology

clim_path = data_dir / "climatology_WB2.nc"
print("Saving climatology to:", clim_path)
if clim_path.exists():
    clim_path.unlink()
rapid_interp.save_climatology(clim_ds_smoothed, clim_path)

In [ ]:
fig_offset, ax_offset = plt.subplots(1, 1, figsize=(8, 5))

for month in clim_ds_smoothed['month'].values:
    dtdp = clim_ds_smoothed['dTdp'].sel(month=month)
    # Offset each month by 0.01 * (month-1)
    offset = 0.01 * (month - 1)
    ax_offset.plot(clim_ds_smoothed['TEMP'], dtdp + offset, label=f'Month {month}')

ax_offset.set_xlabel('Temperature (°C)')
ax_offset.set_ylabel('dTdp (per dbar) + offset')
ax_offset.set_title('Monthly dTdp Climatology with Offset')
ax_offset.legend(ncol=2, fontsize='small')
plt.tight_layout()
plt.show()